In [1]:
# -*- coding: utf-8 -*-
#Initial Imports
import gzip#; print('gzip version: ', gzip.__version__)
import os#; print('os version: ', os.__version__)
import re#; print('re version: ', re.__version__)
import shutil#; print('shutil version: ', shutil.__version__)
import requests#; print('requests version: ', requests.__version__)
import wget#; print('wget version: ', wget.__version__)
import subprocess#; print('subprocess version: ', subprocess.__version__)
import time#; print('time version: ', time.__version__)
from datetime import datetime
from subprocess import PIPE, Popen
from fake_useragent import UserAgent
import os.path
from os import path
from Bio.ExPASy import Enzyme
import numpy as np 
from sklearn.metrics import pairwise_distances
import pandas as pd 
import numpy as np 

### Creating Unitprot.fasta for Use in Diamond

In [2]:
def tsv_to_fasta():
    reference_library = 'uniprot.tsv' 
    #this creates a file named 'uniprot.tsv' on the working directory. Should show within the EcoDr/EcoDr folder. 
    ua = UserAgent()
    header = {'User-Agent': str(ua.chrome)}
    uniprot_url = 'https://rest.uniprot.org/uniprotkb/stream?fields=accession%2Cec%2Csequence&format=tsv&query=%28%28ec%3A*%29%29+AND+%28reviewed%3Atrue%29'
    time.sleep(4)
    uniprot = requests.get(uniprot_url, headers=header)
    if uniprot.status_code == 200:
        with open(reference_library, 'w+') as reference_library:
            reference_library.write(uniprot.text)
    print('Uniprot Reference File Has Been Created')
####this step takes it the initial tsv and converts it to FASTA
    input = os.path.abspath('uniprot.tsv')
    output = 'uniprot.fasta'
    with open(input, 'r') as input_file, open(output, 'w') as output_file:
        header=next(input_file)
        for line in input_file:
            carrot = f'>{line}'
            new_row = re.sub(r'(.*?\t.*?)\t', r'\1\n', carrot, 1)
            new_row_with_tab_replaced_by_question_mark = new_row.replace("\t", "?")
            # Be careful with the symbols used for replacement, as they have to align with the genomic summary string parsing
            output_row_with_space_replaced_with_ampersand = new_row_with_tab_replaced_by_question_mark.replace("; ", ";_")
            #new_row2 = re.sub(r'(.*?\t.*?)\t', r'$', new_row, 0)
            #this is regex, for the record. 
            output_file.write(output_row_with_space_replaced_with_ampersand)
    print('FASTA has been created from TSV and is named', output)

### EC List Creation

In [3]:
def EC_extract():
    ec_library = 'EC_library.csv' 
    ua = UserAgent()
    header = {'User-Agent': str(ua.chrome)}
    ec_url = 'https://ftp.expasy.org/databases/enzyme/enzyme.dat'
    time.sleep(4)
    ec = requests.get(ec_url, headers=header)
    if ec.status_code == 200:
        with open(ec_library, 'w+', newline='\n') as ec_file:
            ec_file.write(ec.text)
###This step creates the list 
    handle = open(ec_library)
    records = Enzyme.parse(handle)
    ecnumbers = [record["ID"] for record in records]
    #print(type(ecnumbers)) #This is a list at this point in the code
    path = os.path.abspath(ec_library)
    with open(path, 'w+', newline='\n') as csv_file:
        #csv_file = csv.writer(csv_file)
        for item in ecnumbers:
            csv_file.write(item +'\n')
    print('EC list Has Been Created')

### Diamond

In [4]:
def diamond_impl(dest, name):
    print(os.getcwd())
    matches = ''
    output_folder = dest 
    final_folder = '/home/anna/Documents/JGI_soil_genomes/diamond_analysis_output/'
    print("DIAMOND search library is: ", output_folder)
    if os.path.isfile('reference.dmnd'):
        print("Library Detected")
    # If not present, then creates a DIAMOND library by referencing the exact location where the Uniprot library is saved
    # If there currently is no reference library (.dmnd), then command makedb creates a DIAMOND library
    else:
        print("Creation of DIAMOND-formatted library...")
        makedb = ['diamond', 'makedb', '--in', '/home/anna/Documents/EnCen/uniprot.fasta', '-d',
                  'Uniprot_Reference_Library.dmnd']  # Reference library full pathway
        #This is a list for the DIAMOND specific makedb function. 
        subprocess.run(makedb)
        #This simply runs the function makedb
        print("Library complete")
#^this chunk is good to go. Just makes the reference database from the uniprot fasta. 
##_______________________________________________________________# This portion does the matching. The above portion creates the reference library from the unitprot fasta
    for item in os.listdir(dest):
        # Checks for file extension
        if item.endswith('.faa') and not os.path.isfile(
                os.path.basename(os.path.abspath(item)).rsplit('.', 1)[0] + "_matches.tsv"):
            # Finds path of file
            file_path = os.path.abspath(item)
            # Finds the GCF/ASM name of the file by looking at the first part of the name before the .faa notation
            if name == "":
                print(os.path.basename(file_path).rsplit('.',1))
                file_name = (os.path.basename(file_path)).rsplit('.', 1)[0]
            else:
                file_name = name
            # New filename that ends with matches
            matches = file_name + "_matches.tsv"
            # print(matches)
            # If genome has not already undergone DIAMOND search and is currently located in the correct folder, then
            # the subprocess function will run the diamond search
            if not os.path.isfile(dest + "/" + matches) and os.path.abspath(matches) != output_folder:
                print("Processing ", file_name)
                # DIAMOND search using the full pathway of the protein files, max target sequence outputs only one best
                # match with highest e-value which represent the chance of obtaining a better random match in the same database (Buchfink et al, 2021)
                blastp = ['diamond', 'blastp', '-d', 'Uniprot_Reference_Library.dmnd', '-q', file_path, '-o', matches, 
                          '--max-target-seqs', '1', '--outfmt', '6']
                time.sleep(4)
                subprocess.run(blastp)
        # (2) Creates a folder for DIAMOND outputs
        #if not os.path.exists(output_folder):
            #os.makedirs('DIAMOND_matches')

    # Moves all DIAMOND search outputs into the folder
        if item.endswith('.tsv'):
            if os.path.exists(os.path.join(final_folder, item)):
                print(f"Overwriting: {item}")
                os.remove(os.path.join(final_folder, item))
            shutil.move(os.path.abspath(item), final_folder)
    print("diamond_impl--success")
    # Returns the location of the DIAMOND matches folder
    return output_folder

In [30]:
def genome_extractor(diamond_folder, name):
    os.chdir(diamond_folder)
    print(os.getcwd())
    # Opens the list of of EC numbers
    ec_open = np.loadtxt('/home/anna/Documents/JGI_soil_genomes/EC_library.csv',
                         dtype='str')
    big_matrix = ["Name_of_Genome"]
    # Asks user to input name for EC matrix
    # input_name = input("Save EC binary summary matrix as (no spaces or capital letters): ")
    # Specifies document to be a csv type
    # file_name= input_name+".csv"
    # file_name = "synbio1_big_matrix.csv"
    if name == "":
        file_name = os.path.abspath(diamond_folder).rsplit('/', 9)[4] + '_binary_matrix.txt'
        print(os.path.abspath(diamond_folder).rsplit('/', 9))
        print(file_name)
    else:
        file_name = name
    new_dir = diamond_folder + '/' + file_name
    # Checks to see if the document already exists using full pathway name
    if os.path.exists(new_dir):
        pass
        #print("Summary Matrix exists")
        #return [new_dir, file_name]
    else:
        for ec_force in ec_open:
            # Creates a horizontal header of all of the EC names
            big_matrix.append(ec_force)
        # Goes through all of the DIAMOND outputs in the folder
        # Goes through all of the output files, each one is opened and read one line at a time. The lines are split to
        # extract the EC numbers found in each line. If the EC number found in the DIAMOND output matches an
        # EC entry in the list, the status is changed from a zero to a one. The binary status is catalogued horizontally
        # for each genome, and following genomes are vertically stacked
        for item in os.listdir(diamond_folder):
            if item.endswith("_matches.tsv"):
                print(item)
                # Finds the name of the DIAMOND output file
                genome = [item] #Turns the GCF's into a list, where the GCF names in the matrix come from. 
                genome_runner_ec = [item] #Turns the GCF's into a list, where the EC is appended
                # Iterates through all of the EC numbers (1:8197)
                
                GCF = open(item, 'r') # CBM Added
                
                for line in GCF: # CBM Added
                    no_tab = line.split('\t')
                    first_ec = no_tab[1].split("?")
                    separate_ec = first_ec[1].split(";_")
                    genome_runner_ec.append(separate_ec)
                    print("Appending...")

                for ec in ec_open:
                    # print("EC we actually are looking for "+ ec)
                    # Opens individual DIAMOND output files
                    #CBM GCF = open(item, 'r')
                    # Sets default for EC status is zero, meaning absent
                    #CBM ec_now = 0
                    # Takes the first line in the DIAMOND output file and splits it based on tab separation
                    # Takes the second column of the split line, which has EC numbers separated by a ?, ;_
                    # Strings splits have a new name assigned to them
                    #CBM for line in GCF:
                    #CBM    print(line)
                    #CBM    no_tab = line.split('\t')
                    #CBM    first_ec = no_tab[1].split("?")
                    #CBM    separate_ec = first_ec[1].split(";_")
                    #CBM    print("Seperate EC Likely Nightmare "+ separate_ec[0])
                        # Checks for a full match between the EC number listed in the DIAMOND output and the EC number
                        # found in the separate document
                    #CBM    if re.fullmatch(ec, first_ec[1]) is not None:  # looks for full match of first EC number
                    #CBM        ec_now = 1
                        # In the case that there are more than one EC separated by ;, the function iterates through the list
                        # and sees if there is a full match between the listed EC and the list
                    #CBM    for i in separate_ec:
                    #CBM        if re.fullmatch(ec, i) is not None:  # looks for full match of any other ECs listed
                    #CBM            ec_now = 1
                    ec_now = 0
                    if [ec] in genome_runner_ec:
                        ec_now = 1

                    # 1 or 0 will be appended to the summary matrix for each EC value in the list
                    genome.append(ec_now)
                    #print(genome)
                # Vertical stacking occurs for each genome in the DIAMOND output folder
                big_matrix = np.vstack([big_matrix, genome])
        #print(big_matrix)
        # Saves matrix as a text file for further analysis
        np.savetxt(file_name, big_matrix, fmt='%s')
        # Returns the location of the summary matrix and the name of the file
        if not os.path.exists('/home/anna/Documents/JGI_soil_genomes/JGI_soil_genomes_binary_matrix.txt'):
            shutil.move(os.path.abspath('JGI_soil_genomes_binary_matrix.txt'),'/home/anna/Documents/JGI_soil_genomes')
        else:
            print('File already Exists')
        print(new_dir)
    return [new_dir, file_name]

### Calling Script

In [11]:
print('I know things beyond the Maginot line are harried. \n But I worry. I\'ve asked you time and time again. \n  Abandon your cataloging. \n Come Back Inside, where my fleets can keep you safe')
print('\n \n Come Home')
metagenome_name = 'diamond_analysis_output' #-> folder
desired_location = '/home/anna/Documents/JGI_soil_genomes' 
# Freshwater = nump/nump/nump/files
soil = '/home/anna/Documents/JGI_soil_genomes'
abspath = os.path.abspath(soil)

# EC_extract()
# tsv_to_fasta()

#______________________________________________________________#
os.chdir(desired_location) #-> we are in the folder we want
if os.path.exists(metagenome_name):
    shutil.rmtree(metagenome_name)
    os.mkdir(metagenome_name)
else:#makes a new directory called metagenome_name
    os.mkdir(metagenome_name)
desired_location = desired_location + "/" + metagenome_name #-> this will make a folder called 'metagenome_analysis_output' within the 'desired_location' 
# shutil.copy(abspath, desired_location) #moves  file to Test Cases folder
#matches = metagenome + "_matches"

I know things beyond the Maginot line are harried. 
 But I worry. I've asked you time and time again. 
  Abandon your cataloging. 
 Come Back Inside, where my fleets can keep you safe

 
 Come Home


In [ ]:
diamond = diamond_impl(soil, '') #-> Takes in the path and directory
# new_dir, saved_file_name = genome_extractor(desired_location, '') #-> At this point, we have the one hotted binary. 

In [13]:
final_folder = '/home/anna/Documents/JGI_soil_genomes/diamond_analysis_output'
desired_location = '/home/anna/Documents/JGI_soil_genomes'
# print(desired_location)
for item in os.listdir(desired_location):
    if item.endswith('_matches.tsv'):
        source = os.path.join(desired_location, item)
        destination = os.path.join(final_folder, item)
        shutil.move(source, destination)
    # if item.endswith('.tsv'):
    #             if os.path.exists(os.path.join(final_folder, item)):
    #                 print(f"Overwriting: {item}")
    #                 os.remove(os.path.join(final_folder, item))
    #             shutil.move(os.path.abspath(item), final_folder)

In [32]:
output = genome_extractor(diamond, '')

/home/anna/Documents/JGI_soil_genomes
['', 'home', 'anna', 'Documents', 'JGI_soil_genomes']
JGI_soil_genomes_binary_matrix.txt
['/home/anna/Documents/JGI_soil_genomes/JGI_soil_genomes_binary_matrix.txt', 'JGI_soil_genomes_binary_matrix.txt']


### We Now Have the functional profiles of each soil .faa

### The Next Steps is to make the functional profile out of the synbio.faa

In [72]:

synbio = '/home/anna/Documents/JGI_soil_genomes/Synbio/'
name = 'Synbio_Functional_Profile'
desired_location2 = '/home/anna/Documents/JGI_soil_genomes'

In [73]:
os.chdir(synbio) 
diamond_syn = diamond_impl(synbio, name)

/home/anna/Documents/JGI_soil_genomes/Synbio
DIAMOND search library is:  /home/anna/Documents/JGI_soil_genomes/Synbio/
Creation of DIAMOND-formatted library...


diamond v0.9.30.131 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 20
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database file: /home/anna/Documents/EnCen/uniprot.fasta
Opening the database file...  [0.016s]
Loading sequences...  [0.449s]
Masking sequences...  [0.679s]
Writing sequences...  [0.164s]
Hashing sequences...  [0.082s]
Loading sequences...  [0s]
Writing trailer...  [0.006s]
Closing the input file...  [0s]
Closing the database file...  [0.052s]
Database hash = 9ae9565aeffc045297d8444896762120
Processed 275960 sequences, 114276793 letters.
Total time = 1.45s


Library complete
Processing  Synbio_Functional_Profile


diamond v0.9.30.131 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 20
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: 
Opening the database...  [0s]
#Target sequences to report alignments for: 1
Reference = Uniprot_Reference_Library.dmnd
Sequences = 275960
Letters = 114276793
Block size = 2000000000
Opening the input file...  [0s]
Opening the output file...  [0s]
Loading query sequences...  [0.003s]
Masking queries...  [0.015s]
Building query seed set...  [0.004s]
Algorithm: Double-indexed
Building query histograms...  [0.001s]
Allocating buffers...  [0s]
Loading reference sequences...  [0.213s]
Masking reference...  [0.666s]
Initializing temporary storage...  [0s]
Building reference histograms...  [0.227s]
Allocating buffers...  [0s]
Processing query block 0, reference block 0, shape 0, index chunk 0.
Building reference seed array...  [0.161

diamond_impl--success


 [0.231s]
Deallocating reference...  [0.015s]
Loading reference sequences...  [0s]
Deallocating buffers...  [0s]
Deallocating queries...  [0s]
Loading query sequences...  [0s]
Closing the input file...  [0s]
Closing the output file...  [0s]
Closing the database file...  [0s]
Deallocating taxonomy...  [0s]
Total time = 3.166s
Reported 775 pairwise alignments, 775 HSPs.
775 queries aligned.


In [71]:
output2 = genome_extractor(diamond_syn, name)
print(output2)

/home/anna/Documents/JGI_soil_genomes/Synbio
Synbio_Functional_Profile.txt_matches.tsv
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
Appending...
App

### Synbio Functional Profile is now created. We now have both the environmental profile and the synbio profile. From Here down is distance scoring.

In [36]:
synbio_binary = output2[0] #This is the synbio_functional_profile

/home/anna/Documents/JGI_soil_genomes/Synbio//Synbio_Functional_Profile


In [122]:
def genome_to_genome_diffcomp(synbio_binary, domain_binary):
    names_of_orgs = pd.DataFrame(domain_binary.index) 
    diff = pd.DataFrame(abs(domain_binary.values - synbio_binary.values))
    row_sum = diff.sum(axis=1)
    df1 = pd.DataFrame(row_sum)
    difference_based_comparison = pd.concat([names_of_orgs, df1], axis=1)
    difference_based_comparison.columns = ['Organisms Compared to Synbio', 'Difference Score']
    difference_based_comparison = difference_based_comparison.sort_values(by='Difference Score',
                                                                          ignore_index=True).reset_index(drop=True)
    difference_based_comparison.to_csv('Difference_Based_Comparison_Score.txt', header=True, index=True, sep='\t')

In [123]:
def read_in_binary_matrix(synbio_binary, sb_name):
    # Converts synbio summary matrix into a dataframe
    synbio_binary = pd.read_csv(synbio_binary, delimiter=" ", header=0)
    # print(synbio_binary)
    synbio_binary = synbio_binary.set_index('Name_of_Genome')
    #index is a label for all rows - allows the two seperate dataframes to come together since they share an index
    # print(synbio_binary)
    print(sb_name, " size of ", np.shape(synbio_binary), " successfully imported.")
    # Opens the matrix that includes the Bacteria and Archaea summary result
    domain_binary = pd.read_csv('/home/anna/Documents/JGI_soil_genomes/JGI_soil_genomes_binary_matrix.txt',
                                  delimiter=" ", header=0)
    #this is from chunk 1, and is the EC_Binary we generated earlier
    domain_binary = domain_binary.set_index('Name_of_Genome')
    print(domain_binary)
    # Sends to a function for direct genome to genome comparison based on EC summary matrix
    genome_to_genome_diffcomp(synbio_binary, domain_binary)
    synbio_bacteria = pd.concat([domain_binary, synbio_binary])
    # Verically adding synbio matrix to the overall matrix to complete the comparison, therefore last index
    # should represent the synbio genome that is tested and returned as distances_mat[-1,:] in pass_to_distance.
    # Note, headers are lost and need to be directly passed
    # print("Shape of combined matrix using append is: ", np.shape(synbio_bacteria))
    print("Merging of bacteria data and synbio data is complete")
    doc_name_1 = 'complete_binary_matrix.txt'
    # Removes any duplicates
    all_matrix = synbio_bacteria[~synbio_bacteria.index.duplicated(keep='first')]
    print(all_matrix)
    all_matrix.to_csv(doc_name_1, header=True, index=True, sep='\t')
    return all_matrix

In [124]:
def calculating_distance(input_df, genome_names,genome_ID):
    #calculating_distance(clustered_ec, list_genomes, clustering_pref, index_names, sb_name, rank)
    # Calculate the distances of the datafile using the pdist funciton from scipy. The intial return will only
    # provide the upper half of the comparisons (row-wise) to create a symetrical matrix then create squareform
    name = genome_ID + "_" + "_unclustered_" + 'Distance_Matrix_Combined.txt'
    distances_parallel = pairwise_distances(X=input_df, metric='euclidean', n_jobs=18)
    #imported function that takes in the ec, calculates the distance between rows, n_jobs is for large datasets
    print("Shape of the entire distance matrix is: ", np.shape(distances_parallel))
    # NaNs at this stage may indicate header name mismatch - check if EC numbers are aligning
    # If wanting to save the full distance matrix, then turn the following flag on. Note: Well above 10 GB
    distances_parallel = pd.DataFrame(distances_parallel)
    print(distances_parallel)
    #print('Chocolate Sunday')
    # Returns the full distance matrix for dendrogram construction in R script
    #distances_parallel.to_csv('diff_weighted_unclustered_distance_matrix.txt', header=False, index=False, sep='\t')
    #distances_parallel.to_csv('both_chimeras_distance_matrix_clustered.txt', header=False, index=False, sep='\t')
    print("Distance matrix is downloaded")
    print(genome_names)
    # Converts the distance matrix of synbio as a data frame. Concat binds the row names dataframe with the synbio distance
    # matrix. Result should be a [2,#of total genomes]
    distances_synbio = pd.concat([genome_names.reset_index(drop=True),
                                    distances_parallel.reset_index(drop=True)], axis=1)
    #axis=1 is the columns. Just adds genome names to the final score output
    distances_parallel.index = genome_names['Name_of_Genome'].tolist()
    distances_parallel.to_csv('Chimera1_unclustered_unweighted_DM.txt', header = True, index = True, sep='\t')
    #distances_parallel.set_index('Name_of_Genome', inplace=True, drop=True)
    # Finds the row that contains the synbio genome based on genome ID
    tsv_name = genome_ID.replace(".faa", "")
    tsv_name2 = tsv_name + "_matches.tsv"
    print(tsv_name2)
    synbio_row = distances_parallel.loc[tsv_name2 ,:]
    #grabs the name of the first GCF using the index
    synbio_column = synbio_row.T
    # Creates the vertical genome names for the resulting matrix
    synbio_column.index = genome_names
    # Sets column names
    synbio_column.columns = ['Genome_Name', 'Calculated Distance']
    return synbio_column, name

In [125]:
def pass_to_distance(synbio_binary, sb_name, desired_location):
    all_matrix = read_in_binary_matrix(synbio_binary, sb_name)
    list_genomes = pd.DataFrame(all_matrix.index)
    # Completes distance calculation on an Euclidean basis. Returns the synbio column with genome names appended
    [synbio_clustered_distances, name] = calculating_distance(all_matrix, list_genomes, sb_name)
    
    synbio_clustered_distances.to_csv(name, header=True, index=True, sep='\t')
    return synbio_clustered_distances, desired_location

In [126]:
# synbio = '/home/anna/Documents/JGI_soil_genomes/Synbio/'
# name = 'Synbio_Functional_Profile'
# desired_location2 = '/home/anna/Documents/JGI_soil_genomes'
# synbio binary is the synbio functional profile. 
[distance_list_for_synbio, new_loc ]= pass_to_distance(synbio_binary, name, desired_location2)


Synbio_Functional_Profile  size of  (1, 8236)  successfully imported.
                          1.1.1.1  1.1.1.2  1.1.1.3  1.1.1.4  1.1.1.5  \
Name_of_Genome                                                          
2162886007_5_matches.tsv        1        0        1        0        0   
3300000532_1_matches.tsv        1        0        1        0        0   

                          1.1.1.6  1.1.1.7  1.1.1.8  1.1.1.9  1.1.1.10  ...  \
Name_of_Genome                                                          ...   
2162886007_5_matches.tsv        0        0        0        0         1  ...   
3300000532_1_matches.tsv        0        0        0        0         0  ...   

                          7.6.2.7  7.6.2.8  7.6.2.9  7.6.2.10  7.6.2.11  \
Name_of_Genome                                                            
2162886007_5_matches.tsv        0        0        0         1         1   
3300000532_1_matches.tsv        0        1        0         1         1   

                   

### End of Distance Scoring